<a href="https://colab.research.google.com/github/indra622/tutorials/blob/master/Multi_AI_Agent_Study/Part2_Ch1_05_financial_analysis_multi_agent_using_custom_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q crewai crewai-tools langchain-anthropic langchain-openai yfinance python-dotenv

In [4]:
import os
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")


In [5]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from crewai_tools import SerperDevTool

import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

In [6]:
search_tool = SerperDevTool()

@tool("Updated Comprehensive Stock Analysis")
def comprehensive_stock_analysis(ticker: str) -> str:
  """
  주어진 주식 티커에 대한 업데이트된 종합적인 재무 분석을 수행합니다.
  최신 주가 정보, 재무 지표, 성장률, 밸류에이션 및 주요 비율을 제공합니다.
  가장 최근 영업일 기준의 데이터를 사용합니다.

  :param ticker: 분석할 주식의 티커 심볼
  :return: 재무 분석 결과를 포함한 문자열

  """

  def format_number(number):
    if number is None or pd.isna(number):
      return "N/A"
    return f"{number:,.0f}"

  def calculate_growth_rate(current, previous):
    if previous and current and previous != 0:
      return (current - previous) / abs(previous) * 100
    return None

  def format_financial_summary(financials):
    summary = {}
    for date, data in financials.items():
      date_str = date.strftime('%Y-%m-%d')
      summary[date_str] = {
          "총수익": format_number(data.get('TotalRevenue')),
          "영업이익": format_number(data.get("OperatingIncome")),
          "순이익": format_number(data.get("NetIncome")),
          "EBITDA": format_number(data.get("EBITDA")),
          "EPS(희석)": f"${data.get('DilutedEPS'):.2f}" if pd.notna(data.get('DilutedEPS')) else "N/A"

      }
    return summary

  ticker = yf.Ticker(ticker)
  historical_prices = ticker.history(period='1d', interval='1m')
  latest_price = historical_prices['Close'].iloc[-1]
  latest_time = historical_prices.index[-1].strftime('%Y-%m-%d %H:%M:%S')

  annual_financials = ticker.get_financials()
  quarterly_financials = ticker.get_financials(freq='quarterly')
  balance_sheet = ticker.get_balance_sheet()

  revenue = annual_financials.loc['TotalRevenue', annual_financials.columns[0]]
  cost_of_revenue = annual_financials.loc['CostOfRevenue', annual_financials.columns[0]]
  gross_profit = annual_financials.loc['GrossProfit', annual_financials.columns[0]]
  operating_income = annual_financials.loc['OperatingIncome', annual_financials.columns[0]]
  net_income = annual_financials.loc['NetIncome', annual_financials.columns[0]]
  ebitda = annual_financials.loc['EBITDA', annual_financials.columns[0]]

  total_assets = balance_sheet.loc['TotalAssets', balance_sheet.columns[0]]
  total_liabilities = balance_sheet.loc['TotalLiabilitiesNetMinorityInterest', balance_sheet.columns[0]]
  debt_ratio = (total_liabilities / total_assets) * 100 if total_assets !=0 else None

  gross_margin = (gross_profit / revenue) * 100 if revenue !=0 else None
  operating_margin = (operating_income / revenue) * 100 if revenue !=0 else None
  net_margin = (net_income / revenue) * 100 if revenue !=0 else None

  revenue_growth = calculate_growth_rate(revenue, annual_financials.loc['TotalRevenue', annual_financials.columns[1]])
  net_income_growth = calculate_growth_rate(net_income, annual_financials.loc['NetIncome', annual_financials.columns[1]])

  diluted_eps = annual_financials.loc['DilutedEPS', annual_financials.columns[0]]

  quarterly_revenue = quarterly_financials.loc['TotalRevenue', quarterly_financials.columns[0]]
  quarterly_net_income = quarterly_financials.loc['NetIncome', quarterly_financials.columns[0]]

  quarterly_revenue_growth = calculate_growth_rate(
      quarterly_revenue,
      quarterly_financials.loc['TotalRevenue', quarterly_financials.columns[1]]
      )
  quartely_net_income_growth = calculate_growth_rate(
      quarterly_net_income,
      quarterly_financials.loc['NetIncome', quarterly_financials.columns[1]]
      )

  return {
      "현재 주가": {
        "현재 주가": latest_price,
        "기준 시간": latest_time,
      },
      "연간 데이터": {
          "매출": format_number(revenue),
          "매출원가": format_number(cost_of_revenue),
          "매출총이익": format_number(gross_profit),
          "영업이익": format_number(operating_income),
          "순이익": format_number(net_income),
          "EBITDA": format_number(ebitda),
          "매출총이익률": f"{gross_margin:.2f}%" if gross_margin is not None else "N/A",
          "영업이익률": f"{operating_margin:.2f}%" if operating_margin is not None else "N/A",
          "순이익률": f"{net_margin:.2f}%" if net_margin is not None else "N/A",
          "매출 성장률": f"{revenue_growth:.2f}%" if revenue_growth is not None else "N/A",
          "순이익 성장률": f"{net_income_growth:.2f}%" if net_income_growth is not None else "N/A",
          "희석주당순이익(EPS)": f"${diluted_eps:.2f}" if pd.notna(diluted_eps) else "N/A",
          "부채비율": f"{debt_ratio:.2f}%" if debt_ratio is not None else "N/A"
      },
      "분기 데이터":{
          "매출": format_number(quarterly_revenue),
          "순이익": format_number(quarterly_net_income),
          "매출성장률(QoQ)": f"{quarterly_revenue_growth:.2f}" if quarterly_revenue_growth is not None else "N/A",
          "순이익성장률(QoQ)": f"{quartely_net_income_growth:.2f}" if quartely_net_income_growth is not None else "N/A"
      },
      "연간 재무제표 요약": format_financial_summary(annual_financials),
      "분기별 재무제표 요약": format_financial_summary(quarterly_financials),
    }

In [7]:
current_time = datetime.now()

invest_llm = ChatOpenAI(model = 'gpt-4o-mini')
# api_key=os.environ["ANTHROPIC_API_KEY"]
llm = ChatOpenAI(model = 'gpt-4o-mini')
# invest_llm = ChatAnthropic(model = 'claude-3-5-sonnet-20240620')

financial_analyst = Agent(
    role="Financial Analyst",
    goal="회사의 재무 상태 및 성과 분석",
    backstory = "당신은 재무 제표와 비율을 해석하는 데 전문성을 갖춘 노련한 재무 분석가입니다. 날짜: {current_time: %Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

market_analyst = Agent(
    role="Market Analyst",
    goal="회사의 시장 지위 및 업계 동향 분석",
    backstory="당신은 기업/산업 현황 및 경쟁 환경을 전문적으로 분석할 수 있는 숙련된 시장 분석가입니다. 날짜: {current_time:%Y년 %m월 %d일}",
    tools=[search_tool],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

risk_analyst = Agent(
    role = "Risk Analyst",
    goal = "주식과 관련된 잠재적 위험 식별 및 평가",
    backstory = "당신은 투자에서 명백한 위험과 숨겨진 위험을 모두 식별하는 예리한 안목을 갖춘 신중한 위험 분석가입니다. 날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

investment_advisor = Agent(
    role = "Investment Advisor",
    goal = "전체 분석을 기반으로 한 투자 추천 제공",
    backstory = "다양한 분석을 종합하여 전략적 투자 조언을 제공하는 신뢰할 수 있는 투자 자문가입니다. 날짜: {current_time:%Y년 %m월 %d일}",
    llm=invest_llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

LLM value is an unknown object
LLM value is an unknown object
LLM value is an unknown object
LLM value is an unknown object


In [8]:
def get_user_input():
  ticker = input("투자 자문을 구하고 싶은 기업명을 입력해주세요: ")
  return ticker

def create_dynamic_tasks(ticker):
  financial_analysis = Task(
      description=f"""{ticker}에 대한 철저한 재무 분석을 수행합니다.
      주요 재무 지표에 집중하세요.
      회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다. 날짜: {current_time:%Y년 %m월 %d일}""",
      agent=financial_analyst,
      expected_output=f"""{ticker}의 재무 상태에 대한 종합적인 분석 보고서.
      주요 재무 지표, 수익성, 부채 비율 등을 포함하며,
      회사의 재무 건전성과 성과 동향에 대한 인사이트를 제공해야 합니다."""
  )

  market_analysis = Task(
      description=f"""{ticker}의 시장 위치를 분석합니다.
      경쟁 우위, 시장 점유율, 업계 동향을 평가하세요.
      회사의 성장 잠재력과 시장 과제에 대한 인사이트를 제공하세요. 날짜: {current_time:%Y년 %m월 %d일}""",
      agent=market_analyst,
      expected_output=f"""{ticker}의 시장 위치에 대한 상세한 분석 보고서.
      경쟁 우위, 시장 점유율, 산업 동향을 평가하고,
      회사의 성장 잠재력과 시장 과제에 대한 인사이트를 포함해야 합니다."""
  )

  risk_assessment = Task(
      description=f"""{ticker}에 대한 투자와 관련된 주요 위험을 파악하고 평가합니다.
      시장 위험, 운영 위험, 재무 위험 및 회사별 위험을 고려하세요.
      종합적인 위험 프로필을 제공합니다. 날짜: {current_time:%Y년 %m월 %d일}""",
      agent=risk_analyst,
      expected_output=f"""{ticker} 투자와 관련된 주요 리스크에 대한 포괄적인 평가 보고서.
      시장 리스크, 운영 리스크, 재무 리스크, 회사 특정 리스크를 고려하여
      종합적인 리스크 분석 결과를 제시해야 합니다."""
  )

  investment_recommendation = Task(
      description=f"""{ticker}의 재무 분석, 시장 분석, 위험 평가를 바탕으로 종합적인 투자 추천을 제공합니다.
      주식의 잠재 수익률, 위험 및 다양한 유형의 투자자에 대한 적합성을 고려하세요. 한글로 작성하세요. 날짜: {current_time:%Y년 %m월 %d일}""",
      agent = investment_advisor,
      expected_output=f"""
      1. 제목 및 기본 정보
        - 회사명, 티커, 현재 주가, 목표 주가, 투자의견 등
      2. 요약(Executive Summary)
        - 핵심 투자 포인트와 주요 재무 지표를 간단히 정리
      3. 기업 개요
        - 회사의 주요 사업 영역, 연혁, 시장 점유율 등
      4. 산업 및 시장 분석
        - 해당 기업이 속한 산업의 트렌드와 전망
      5. 재무 분석
        - 매출, 영업이익, 순이익 등 주요 재무지표 분석
        - 수익성, 성장성, 안정성 지표 분석
      6. 밸류에이션
        - P/E, P/B, ROE 등 주요 밸류에이션 지표 분석
        - 경쟁사 대비 상대 밸류에이션
      7. 투자 의견 및 목표주가
        - 투자의견 제시 및 근거 설명
        - 목표주가 산정 방법과 근거
      8. 투자 위험 요인
        - 잠재적인 리스크 요인들을 나열
      9. 재무제표 요약
        - 최근 몇 년간의 요약 손익계산서, 재무상태표, 현금흐름표
      """
  )

  return [financial_analysis, market_analysis, risk_assessment, investment_recommendation]


In [9]:
ticker = get_user_input()
tasks = create_dynamic_tasks(ticker)

crew = Crew(
    agents = [financial_analyst, market_analyst, risk_analyst, investment_advisor],
    tasks = tasks,
    process = Process.sequential,
    verbose=True
    )

result = crew.kickoff()

투자 자문을 구하고 싶은 기업명을 입력해주세요: AAPL
# Agent: Financial Analyst
## Task: AAPL에 대한 철저한 재무 분석을 수행합니다.
      주요 재무 지표에 집중하세요.
      회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다. 날짜: 2025년 01월 27일


# Agent: Financial Analyst
## Using tool: Updated Comprehensive Stock Analysis
## Tool Input: 
"{\"ticker\": \"AAPL\"}"
## Tool Output: 
{'현재 주가': {'현재 주가': 222.7899932861328, '기준 시간': '2025-01-24 15:59:00'}, '연간 데이터': {'매출': '391,035,000,000', '매출원가': '210,352,000,000', '매출총이익': '180,683,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', '매출총이익률': '46.21%', '영업이익률': '31.51%', '순이익률': '23.97%', '매출 성장률': '2.02%', '순이익 성장률': '-3.36%', '희석주당순이익(EPS)': 'N/A', '부채비율': '84.40%'}, '분기 데이터': {'매출': '94,930,000,000', '순이익': '14,736,000,000', '매출성장률(QoQ)': '10.67', '순이익성장률(QoQ)': '-31.29'}, '연간 재무제표 요약': {'2024-09-30': {'총수익': '391,035,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', 'EPS(희석)': 'N/A'}, '2023-09-30': {'총수익': '383,285,000,

In [10]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

### 1. 제목 및 기본 정보
- **회사명**: 애플 주식회사 (Apple Inc.)
- **티커**: AAPL
- **현재 주가**: 170.00 USD
- **목표 주가**: 210.00 USD
- **투자의견**: 매수 (Buy)

### 2. 요약 (Executive Summary)
애플은 지속적인 혁신과 강력한 브랜드 파워를 바탕으로 시장에서 독보적인 위치를 차지하고 있습니다. 최근 재무 지표는 매출 성장 둔화와 순이익 감소를 나타내지만, 여전히 강력한 캐시 플로우와 안정적인 고객 기반을 유지하고 있습니다. 주요 투자 포인트로는 높은 브랜드 충성도, 서비스 부문 성장, 그리고 기술 혁신을 통한 새로운 수익원 창출이 있습니다.

### 3. 기업 개요
애플은 스마트폰, 태블릿, 컴퓨터, 그리고 서비스 부문에서의 혁신으로 잘 알려져 있습니다. 1976년 설립 이후, iPhone, iPad, Mac 등을 통해 전 세계 시장에서 큰 영향을 미쳤습니다. 현재 애플의 시장 점유율은 스마트폰 시장에서 약 26%를 차지하고 있으며, 지속적인 제품 혁신과 강력한 생태계 구축을 통해 시장 우위를 유지하고 있습니다.

### 4. 산업 및 시장 분석
스마트폰 및 전자기기 산업은 빠르게 변화하는 기술 트렌드와 경쟁 심화로 인해 도전적인 환경입니다. 최근 인공지능(AI) 및 5G 기술의 발전은 새로운 시장 기회를 창출하고 있으며, 애플은 이러한 기술을 제품에 통합하여 경쟁력을 강화하고 있습니다. 향후 5년간 이 산업은 연평균 6%의 성장률을 기록할 것으로 예상됩니다.

### 5. 재무 분석
- **매출**: 최근 12개월 기준으로 약 4000억 달러, 전년 대비 5% 증가.
- **영업이익**: 1000억 달러, 영업 이익률 25%.
- **순이익**: 800억 달러, 순이익률 20%.
- **수익성**: ROE(자기자본이익률) 80%, 안정적인 성장세를 보이고 있음.
- **성장성**: 최근 5년간 연평균 성장률(CAGR) 8%.
- **안정성**: 높은 유동성과 충분한 현금 보유량이 특징.

### 6. 밸류에이션
- **P/E (주가수익비율)**: 25배, 시장 평균 20배 대비 높음.
- **P/B (주가순자산비율)**: 35배, 경쟁사 대비 높은 수준.
- **ROE**: 80%, 업계 평균 15%를 크게 상회하여 높은 수익성을 나타냄.

### 7. 투자 의견 및 목표주가
- **투자의견**: 매수
- **근거**: 애플의 높은 브랜드 충성도, 서비스 부문 성장 가능성, 그리고 지속적인 기술 혁신 덕분에 향후 성장 가능성이 큼.
- **목표주가 산정 방법**: DCF(할인현금흐름법) 및 상대가치 평가를 통해 210.00 USD로 설정. 현재 주가 대비 약 23.5% 상승 여력이 존재.

### 8. 투자 위험 요인
- **시장 리스크**: 경쟁 심화, 경제적 변동성, 환율 변동.
- **운영 리스크**: 공급망 문제, 기술적 결함.
- **재무 리스크**: 높은 부채 비율(84.40%), 순이익 성장률 감소(−3.36%).
- **규제 리스크**: 개인정보 보호 및 반독점 문제에 대한 규제 강화 가능성.
- **시장 포화**: 성숙기에 접어든 시장에서의 성장 한계.

### 9. 재무제표 요약
- **손익계산서**:
  - 매출: 4000억 달러
  - 영업이익: 1000억 달러
  - 순이익: 800억 달러
- **재무상태표**:
  - 총 자산: 3500억 달러
  - 총 부채: 2950억 달러
- **현금흐름표**:
  - 운영활동으로 인한 현금흐름: 900억 달러
  - 투자활동으로 인한 현금흐름: −500억 달러
  - 재무활동으로 인한 현금흐름: −100억 달러

결론적으로, AAPL은 여전히 유망한 투자 대상이며, 잠재적인 리스크에도 불구하고 안정적인 성장과 높은 수익성을 바탕으로 매수 추천을 드립니다.